In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_seq_items', 100)

# 400 meter coordinate accuracy
# 1923 earliest year

In [ ]:
# read in new merged gbif_obis data
new_path = input("Enter the file path: ")

new = pd.read_parquet(new_path)

In [ ]:
new

In [ ]:
# read in old species data
old_path = input("Enter the file path: ")

# read in the old species data
# old = pd.read_csv(old_path)
old = pd.read_parquet(old_path)

In [ ]:
# old["observed_ecoregions"].iloc[0][0]

In [ ]:
# only necessary if not already done(if reading parquet its already done)

# this has to be done because of bad formatting when downloading from db, might be a better way to download or import. But for now will have to be done for each column that contains a list. THIS IS SLOW, 20+ MIN FOR 300,000+ ROWS
for i in range(len(old)):
    print(i)
    old["observed_ecoregions"].iloc[i] = (
        old["observed_ecoregions"]
        .iloc[i]
        .replace('["', "")
        .replace('"]', "")
        .split('","')
    )

In [ ]:
# only necessary if not already done(if reading parquet its already done)

# this has to be done because of bad formatting when downloading from db, might be a better way to download or import. But for now will have to be done for each column that contains a list. THIS IS SLOW, 20+ MIN FOR 300,000+ ROWS

for i in range(len(old)):
    print(i)
    # handle missing values (missing values are type float)
    if type(old["native_ecoregions"].iloc[i]) == str:
        old["native_ecoregions"].iloc[i] = (
            old["native_ecoregions"]
            .iloc[i]
            .replace('["', "")
            .replace('"]', "")
            .split('","')
        )
    else:
        print("missing")
        old["native_ecoregions"].iloc[i] = None

In [ ]:
old

In [ ]:
# only necessary if not already done(if reading parquet its already done)

old.to_parquet(
    "/media/muskrat/T7 Shield/eco_data/v4/db_exports/species_formatted.parquet"
)

In [ ]:
old_main = old[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "observed_ecoregions",
        # "rights",
    ]
]

old_main

In [ ]:
# create column of old_main called rights where value of every row equals [None]
old_main["rights"] = [None] * len(old_main)

# need to make placeholders in rights for each ecoregion
for i in range(len(old_main)):
    eco_length = len(old_main["observed_ecoregions"].iloc[i])
    # old_main["rights"].iloc[i] equals array of None with length of eco_length
    old_main["rights"].iloc[i] = [None] * eco_length

old_main

In [ ]:
side_piece = old[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "native_ecoregions",
    ]
]

side_piece

In [ ]:
combined = pd.concat([new, old_main], axis=0)

combined

In [ ]:
del new, old, old_main

In [ ]:
combined_grouped = combined.groupby(
    ["scientific_name", "kingdom", "phylum", "class", "order", "family", "genus"],
    as_index=False,
    dropna=False,
).apply(
    lambda x: pd.Series(
        {
            "observed_ecoregions": list(
                [item for sublist in x["observed_ecoregions"] for item in sublist]
            ),
            "rights": list([item for sublist in x["rights"] for item in sublist]),
        }
    )
)

# combined_grouped = combined.groupby(
#     ["scientific_name", "kingdom", "phylum", "class", "order", "family", "genus"],
#     as_index=False, dropna=False,
# ).apply(
#     lambda x: pd.Series(
#         {
#             "observed_ecoregions": list(
#                 set([item for sublist in x["observed_ecoregions"] for item in sublist])
#             ),
#             "rights": list(
#                 set(
#                     [
#                         item
#                         for sublist in x["rights"]
#                         for item in sublist
#                         if item is not None
#                     ]
#                 )
#             ),
#         }
#     )
# )

In [ ]:
print(
    f'max ecoregions length is {combined_grouped["observed_ecoregions"].str.len().max()}'
)
print(f'max rights length is {combined_grouped["rights"].str.len().max()}')

In [ ]:
del combined

In [ ]:
combined_grouped

In [ ]:
# SLOW, TOOK ABOUT AN HOUR FOR 300,000+ ROWS
for i in range(len(combined_grouped)):
    print(f"index: {i}")
    # print(len(test["observed_ecoregions"].iloc[i]))
    cat = []
    rights_cat = []
    for j in range(len(combined_grouped["observed_ecoregions"].iloc[i])):
        if combined_grouped["observed_ecoregions"].iloc[i][j] not in cat:
            cat.append(combined_grouped["observed_ecoregions"].iloc[i][j])
            rights_cat.append(combined_grouped["rights"].iloc[i][j])
        # print(test["observed_ecoregions"].iloc[i][j])
        # for k in test["observed_ecoregions"].iloc[i][j]:
        # print(k)
        # if k not in cat:
        #     cat.append(k)
    # if len(cat) != len(test["observed_ecoregions"].iloc[i]):
    # print(f'index: {i}')

    # print(f'observed: {test["observed_ecoregions"].iloc[i]}')
    # print(f'cat: {cat}')
    if len(combined_grouped["rights"].iloc[i]) != len(
        combined_grouped["observed_ecoregions"].iloc[i]
    ):
        print(f"index: {i}")
        print("rights")

    combined_grouped["observed_ecoregions"].iloc[i] = cat
    combined_grouped["rights"].iloc[i] = rights_cat

In [ ]:
combined_grouped

In [ ]:
combined_grouped.to_parquet(
    "/media/muskrat/T7 Shield/eco_data/v4/occurences/step_4/processing/step_2/merge/old_new_retry_2_init.parquet"
)

In [ ]:
merged = pd.merge(
    combined_grouped,
    side_piece,
    how="outer",
    on=["scientific_name", "kingdom", "phylum", "class", "order", "family", "genus"],
)

merged

In [ ]:
merged.to_parquet(
    "/media/muskrat/T7 Shield/eco_data/v4/occurences/step_4/processing/step_2/merge/old_new_retry_2_final.parquet"
)

In [ ]:
duplicates = merged[merged.duplicated(["scientific_name"], keep=False)]
duplicates

In [ ]:
# OLD CODE BELOW

In [ ]:
species = old["scientific_name"]

In [ ]:
# check for strange x at beginning of species names and remove if necessary

# old = old[
#     ~old.scientific_name.isin(["× Heucherella", "×Elyhordeum schaackianum", "×Elyleymus aleuticus"])
# ]

In [ ]:
# check for species names that longer than 2 words and remove if necessary
species_values = np.array([len(word.split()) for word in species])

# find the max value in species_values
max_value = np.max(species_values)

In [ ]:
# if value at index i in species_values is greater than 2, add the value at the same index in species_unique to a new list
species_2 = []
for i in range(len(species_values)):
    if species_values[i] > 2:
        species_2.append(species[i])

In [ ]:
# remove values from species that are greater than 2

for i in range(len(species_2)):
    old = old[old.scientific_name != species_2[i]]

In [ ]:
del species_2, species_values, max_value, i, species

In [ ]:
inner = pd.merge(new, old, how="inner", on=["scientific_name"], indicator=True)

In [ ]:
inner

In [ ]:
inner["combined_id"] = inner["unique_id_x"].apply(lambda x: x.tolist()) + inner[
    "unique_id_y"
].apply(lambda x: x.tolist())

In [ ]:
inner["combined_id"] = inner["combined_id"].apply(set).apply(list)

In [ ]:
check_combined_id = inner[["unique_id_x", "unique_id_y", "combined_id"]]

In [ ]:
inner = inner.drop(["unique_id_x", "unique_id_y"], axis=1)

inner.rename(columns={"combined_id": "unique_id"}, inplace=True)

del check_combined_id

In [ ]:
inner["combined_rights"] = inner["rights_x"].apply(lambda x: x.tolist()) + inner[
    "rights_y"
].apply(lambda x: x.tolist())

In [ ]:
# check combined_rights
print(
    f"right {inner['rights_x'][0]}, left {inner['rights_y'][0]}, combined {inner['combined_rights'][0]}"
)

In [ ]:
# remove duplicates
rights = inner["combined_rights"].to_list()

In [ ]:
for j in range(len(rights)):
    print(j)
    rights[j] = [i for n, i in enumerate(rights[j]) if i not in rights[j][n + 1 :]]

In [ ]:
rights = pd.Series(rights, name="rights")

inner["combined_rights"] = rights

In [ ]:
# recheck combined_rights
print(
    f"right {inner['rights_x'][493]}, left {inner['rights_y'][493]}, combined {inner['combined_rights'][493]}"
)

In [ ]:
inner = inner.drop(["rights_x", "rights_y"], axis=1)

inner.rename(columns={"combined_rights": "rights"}, inplace=True)

del rights, j

In [ ]:
# keep columns with less missing values

kingdom_x_missing = inner["kingdom_x"].isna().sum()

kingdom_y_missing = inner["kingdom_y"].isna().sum()

# drop the kingdom column from inner that contains more missing values
if kingdom_x_missing > kingdom_y_missing:
    inner = inner.drop(["kingdom_x"], axis=1)
    inner.rename(columns={"kingdom_y": "kingdom"}, inplace=True)
else:
    inner = inner.drop(["kingdom_y"], axis=1)
    inner.rename(columns={"kingdom_x": "kingdom"}, inplace=True)

phylum_x_missing = inner["phylum_x"].isna().sum()

phylum_y_missing = inner["phylum_y"].isna().sum()

# drop the phylum column from inner that contains more missing values

if phylum_x_missing > phylum_y_missing:
    inner = inner.drop(["phylum_x"], axis=1)
    inner.rename(columns={"phylum_y": "phylum"}, inplace=True)
else:
    inner = inner.drop(["phylum_y"], axis=1)
    inner.rename(columns={"phylum_x": "phylum"}, inplace=True)

# find sum of missing values in inner['class_x']
class_x_missing = inner["class_x"].isna().sum()

class_y_missing = inner["class_y"].isna().sum()

# drop the class column from inner that contains more missing values
if class_x_missing > class_y_missing:
    inner = inner.drop(["class_x"], axis=1)
    inner.rename(columns={"class_y": "class"}, inplace=True)
else:
    inner = inner.drop(["class_y"], axis=1)
    inner.rename(columns={"class_x": "class"}, inplace=True)

# check
class_missing = inner["class"].isna().sum()

order_x_missing = inner["order_x"].isna().sum()

order_y_missing = inner["order_y"].isna().sum()

# drop the order column from inner that contains more missing values

if order_x_missing > order_y_missing:
    inner = inner.drop(["order_x"], axis=1)
    inner.rename(columns={"order_y": "order"}, inplace=True)
else:
    inner = inner.drop(["order_y"], axis=1)
    inner.rename(columns={"order_x": "order"}, inplace=True)

family_x_missing = inner["family_x"].isna().sum()

family_y_missing = inner["family_y"].isna().sum()

# drop the family column from inner that contains more missing values

if family_x_missing > family_y_missing:
    inner = inner.drop(["family_x"], axis=1)
    inner.rename(columns={"family_y": "family"}, inplace=True)
else:
    inner = inner.drop(["family_y"], axis=1)
    inner.rename(columns={"family_x": "family"}, inplace=True)


genus_x_missing = inner["genus_x"].isna().sum()

genus_y_missing = inner["genus_y"].isna().sum()

# drop the genus column from inner that contains more missing values

if genus_x_missing > genus_y_missing:
    inner = inner.drop(["genus_x"], axis=1)
    inner.rename(columns={"genus_y": "genus"}, inplace=True)
else:
    inner = inner.drop(["genus_y"], axis=1)
    inner.rename(columns={"genus_x": "genus"}, inplace=True)

In [ ]:
del (
    genus_y_missing,
    genus_x_missing,
    family_y_missing,
    family_x_missing,
    order_y_missing,
    order_x_missing,
    class_y_missing,
    class_x_missing,
    phylum_y_missing,
    phylum_x_missing,
    kingdom_y_missing,
    kingdom_x_missing,
    class_missing,
)

In [ ]:
# drop _merge column
inner = inner.drop("_merge", axis=1)

In [ ]:
inner = inner[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

In [ ]:
outer = pd.merge(new, old, how="outer", on=["scientific_name"], indicator=True)

In [ ]:
outer

In [ ]:
left = outer.loc[
    outer._merge == "left_only",
    [
        "kingdom_x",
        "phylum_x",
        "class_x",
        "order_x",
        "family_x",
        "genus_x",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id_x",
        "rights_x",
    ],
]

In [ ]:
left.rename(
    columns={
        "kingdom_x": "kingdom",
        "phylum_x": "phylum",
        "class_x": "class",
        "order_x": "order",
        "family_x": "family",
        "genus_x": "genus",
        "unique_id_x": "unique_id",
        "rights_x": "rights",
    },
    inplace=True,
)

In [ ]:
left = left[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

In [ ]:
left

In [ ]:
right = outer.loc[
    outer._merge == "right_only",
    [
        "kingdom_y",
        "phylum_y",
        "class_y",
        "order_y",
        "family_y",
        "genus_y",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id_y",
        "rights_y",
    ],
]

In [ ]:
right.rename(
    columns={
        "kingdom_y": "kingdom",
        "phylum_y": "phylum",
        "class_y": "class",
        "order_y": "order",
        "family_y": "family",
        "genus_y": "genus",
        "unique_id_y": "unique_id",
        "rights_y": "rights",
    },
    inplace=True,
)

In [ ]:
right = right[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

In [ ]:
right

In [ ]:
frames = [inner, left, right]

# concatenate dataframes
final = pd.concat(frames)

# reset index
final.reset_index(drop=True, inplace=True)

In [ ]:
final

In [ ]:
# find unique values in scientific_name column of final
species_unique = final["scientific_name"].unique()

In [ ]:
del frames, left, right, inner, outer, new, old, species_unique

In [ ]:
# read cleaned df into a parquet file where user can input the file path
df_merge_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
final.to_parquet(df_merge_path)

In [ ]:
# merged = merged.drop(["unique_id_y", "rights_y", "_merge"], axis=1)

# merged.rename(columns={"unique_id_x": "unique_id",
#               "rights_x": "rights"}, inplace=True)

In [ ]:
# merged

In [ ]:
# merged = merged[
#     [
#         "kingdom",
#         "phylum",
#         "class",
#         "order",
#         "family",
#         "genus",
#         "scientific_name",
#         "common_name",
#         "species_type",
#         "unique_id",
#         "rights",
#     ]
# ]

In [ ]:
# read cleaned df into a parquet file where user can input the file path
df_merge_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
merged.to_parquet(df_merge_path)